# MuFoCo first model

**[MuJoCo](https://mujoco.readthedocs.io/en/stable/overview.html)** stands for Multi-Joint dynamics with Contact. It is a general purpose physics engine that aims to facilitate research and development in many domains, but we have primary interest in robotics. Within this tutorial we are going to create our first simple simulation model.

# Code explanation
Let us start with loading needed libraries. Within this simple example we are going to use `mujoco` and a custom viewer `mujoco_viewer`.

In [1]:
import mujoco
import mujoco_viewer

You can define models in the native MJCF scene description language – an XML file format designed to be as human readable and editable as possible. URDF model files can also be loaded. Within this example we are going to use **[MJCF](https://mujoco.readthedocs.io/en/stable/XMLreference.html)**. 

The following model contains one fixed body composed out of a few geoms and one floating body. 

In [2]:
MJCF_MODEL = """ 
<mujoco>
  <option gravity ="0 0 -9.8" timestep="0.00005"/>

  <worldbody>
    <light diffuse=".5 .5 .5" pos="0 0 3" dir="0 0 -1"/>
    <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
    <geom type="plane" size="1 1 0.1" rgba=".9 0 0 1"/>

    <body name="float_body" pos="0 0 2">
      <joint type="free"/>
      <geom  type="box" size=".1 .2 .3" rgba="0 .9 0 1" mass = "1"/>
    </body>

  </worldbody>
</mujoco>
"""

MuJoCo separates simulation parameters into two data structures at runtime: (1) mjModel contains the model description and is expected to remain constant, and (2) mjData contains all dynamic variables and intermediate results.

In [3]:
model = mujoco.MjModel.from_xml_string(MJCF_MODEL)
data = mujoco.MjData(model)

Here we use a custom mujoco-python-viewer. 

`mj_step` will simulate the passive dynamics, because we have not provided any control signals or applied forces. 

Press `D` to turn off rendering of all frames to force viewer to render in real time. 

In [4]:
viewer = mujoco_viewer.MujocoViewer(model, data)

while viewer.is_alive:
    mujoco.mj_step(model, data)

    viewer.render()

viewer.close()    

You can create `mujoco.MjModel` not from string only, but from a file as well. 

Also, I have added a limit for number of simulation steps.

In [ ]:
from os import path

MODEL_NAME = 'ex_7.xml'
ROOT = path.abspath('')
FOLDER_PATH = path.join(ROOT, 'models')
MODEL_PATH = path.join(FOLDER_PATH, MODEL_NAME)

model = mujoco.MjModel.from_xml_path(MODEL_PATH)
data = mujoco.MjData(model)

viewer = mujoco_viewer.MujocoViewer(model, data)

STEP_NUM = 1000

for i in range(STEP_NUM):
    if viewer.is_alive:
        mujoco.mj_step(model, data)
        viewer.render()
        print(i)
    else:
        break

viewer.close()    

Now let us try the default viewer. Additionally, we have added extra `flags`.

In [6]:
import time
from mujoco import viewer
from os import path

MODEL_NAME = 'ex_7.xml'
ROOT = path.abspath('')
FOLDER_PATH = path.join(ROOT, 'models')
MODEL_PATH = path.join(FOLDER_PATH, MODEL_NAME)

model = mujoco.MjModel.from_xml_path(MODEL_PATH)
data = mujoco.MjData(model)

SIMULATION_TIME = 100

with mujoco.viewer.launch_passive(model, data) as viewer:
    start = time.time()

    while viewer.is_running() and time.time() - start < SIMULATION_TIME:
        mujoco.mj_step(model, data)
        viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_CONTACTPOINT] = True
        viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = True
        viewer.sync()

viewer.close()

As a homework try to modify or change the entire MJCF model. Use **[Overview](https://mujoco.readthedocs.io/en/stable/overview.html#)** and **[XML Reference](https://mujoco.readthedocs.io/en/stable/XMLreference.html)** for your help. Have fun!